# Visualize CryinGAN training

In [ ]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# Read and visualize losses csv

df = pd.read_csv("../CryinGAN/losses.csv").iloc[201:].astype(float).reset_index(drop=True).set_index("epoch")

fig = plt.figure(figsize=(20,10))
sns.lineplot(data=df, figure=fig, marker="o")

In [ ]:
fig = plt.figure(figsize=(20,10))
sns.lineplot(data=df[["cost_gen", "D_fake"]],figure=fig, marker="o")

## Check on the generated structures

In [ ]:
file = "../CryinGAN/gen_data/gen_coords_15.npy"

import numpy as np

np.fromfile(file).shape